# DAC Contest
This reference design will help you walk through a design flow of DAC SDC 2023. This is a simplified design to help users get started on the FPGA platform and to understand the overall flow. It does not contain any object detection hardware.

If you have any questions, please post on the Slack page (link on SDC website sidebar).

### Hardware

### Software
Note:
  * You will not submit your `dac_sdc.py` file, so any changes you make to this file will not be considered during evluation.  
  * You can use both PS and PL side to do inference.

### Object Detection

Object detection will be done on images in batches:
  * You will provide a Python callback function that will perform object detection on batch of images.  This callback function wile be called many times.
  * The callback function should return the locations of all images in the batch.
  * Runtime will be recorded during your callback function.
  * Images will be loaded from SD card before each batch is run, and this does not count toward your energy usage or runtime.
  
### Notebook
Your notebook should contain 4 code cells:

1. Importing all libraries and creating your Team object.
1. Downloading the overlay, compile the code, and performany any one-time configuration.
1. Python callback function and any other Python helper functions.
1. Running object detection
1. Cleanup



## 1. Imports and Create Team

In [1]:
import sys
import os

sys.path.append(os.path.abspath("../common"))
import dac_sdc

'''
we need numpy==1.19.4
'''
import tensorrt as trt
import pycuda.autoinit
import pycuda.driver as cuda
import numpy as np
import cv2

team_name = 'GO3'
dac_sdc.BATCH_SIZE = 1
team = dac_sdc.Team(team_name)

**Your team directory where you can access your notebook, and any other files you submit, is available as `team.team_dir`.**

## 2. Preparing the library and model
Prepare the dependencies for the contest, including installing python packages, compiling your binaries, and linking to the notebook.

Your team is responsible to make sure the correct packages are installed. For the contest environment, use the configuration below provided by Nvidia:
- [JetPack 4.6.1](https://developer.nvidia.com/embedded/jetpack-sdk-461)
    - Ubuntu 18.04
    - CUDA 10.2
    - cuDNN 8.2.1
    - gcc 7.5.0
    - python 3.6.9
    - TensorRT 8.2.1

In [2]:
class BaseEngine(object):
    def __init__(self, engine_path):
        self.n_classes = 10
        logger = trt.Logger(trt.Logger.WARNING)
        logger.min_severity = trt.Logger.Severity.ERROR
        runtime = trt.Runtime(logger)
        trt.init_libnvinfer_plugins(logger, '')  # initialize TensorRT plugins
        with open(engine_path, "rb") as f:
            serialized_engine = f.read()
        engine = runtime.deserialize_cuda_engine(serialized_engine)
        self.imgsz = engine.get_binding_shape(0)[2:]  # get the read shape of model, in case user input it wrong
        self.context = engine.create_execution_context()
        self.inputs, self.outputs, self.bindings = [], [], []
        self.stream = cuda.Stream()
        for binding in engine:
            size = trt.volume(engine.get_binding_shape(binding))
            dtype = trt.nptype(engine.get_binding_dtype(binding))
            host_mem = cuda.pagelocked_empty(size, dtype)
            device_mem = cuda.mem_alloc(host_mem.nbytes)
            self.bindings.append(int(device_mem))
            if engine.binding_is_input(binding):
                self.inputs.append({'host': host_mem, 'device': device_mem})
            else:
                self.outputs.append({'host': host_mem, 'device': device_mem})

    def infer(self, img):
        self.inputs[0]['host'] = img
        for inp in self.inputs:
            cuda.memcpy_htod_async(inp['device'], inp['host'], self.stream)
        self.context.execute_async_v2(
            bindings=self.bindings,
            stream_handle=self.stream.handle)

        for i, out in enumerate(self.outputs):
            if i in [3, 4, 6]:
                cuda.memcpy_dtoh_async(out['host'], out['device'], self.stream)

        self.stream.synchronize()
        data = [out['host'] for out in self.outputs]
        return data

    def inference(self, rgb_img):
        img, ratio = letterbox(rgb_img, (288, 512))
        img = img.astype(np.float32)[None]  # uint8 to fp16/32
        data = self.infer(img)
        _, _, _, num, final_boxes, _, final_cls_inds = data
        final_boxes = np.reshape(final_boxes / ratio, (-1, 4))[:num[0]]
        final_cls_inds = np.array(final_cls_inds)[:num[0]].reshape(-1)
        return final_boxes, final_cls_inds


class Predictor(BaseEngine):
    def __init__(self, engine_path):
        super(Predictor, self).__init__(engine_path)
        self.n_classes = 10  # your model classes


def warmup(pred):
    print('-----------start warmup-----------')
    warmup_dir = './warmup'
    for filename in os.listdir(warmup_dir):
        if filename.lower().endswith('.jpg'):
            image_path = os.path.join(warmup_dir, filename)
            bgr_img = cv2.imread(image_path)
            rgb_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
            _, _ = pred.inference(rgb_img)
    print('-----------warmup finished-----------')
    print('-----------start inference, wait a moment-----------')


def letterbox(im, new_shape=(288, 512)):
    shape = im.shape[:2]  # current shape [height, width]
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    r = min(r, 1.0)
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding

    if shape[::-1] != new_unpad:  # resize
        im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_NEAREST)

    im = cv2.copyMakeBorder(im, 0, dh, 0, dw, cv2.BORDER_CONSTANT, value=(114, 114, 114))  # add border

    return im, r


pred = Predictor(engine_path='model.trt')
warmup(pred)

-----------start warmup-----------
-----------warmup finished-----------
-----------start inference, wait a moment-----------


## 3. Python Callback Function and Helper Functions


### Pushing the picture through the pipeline
In this example, we use contiguous memory arrays for sending and receiving data via DMA.

The size of the buffer depends on the size of the input or output data.  The example images are 640x360 (same size as training and test data), and we will use `pynq.allocate` to allocate contiguous memory.

### Callback function
The callback function:
  - Will be called on each batch of images (will be called many times)
  - Is prvided with a list of tuples of (image path, RGB image)
  - It should return a dictionary with an entry for each image:
    - Key: Image name (`img_path.name`)
    - Value: Dictionary of item type and bounding box (keys: `type`, `x`, `y`, `width`, `height`)

See the code below for an example:


In [3]:
def my_callback(rgb_imgs):
    object_locations_by_image = {}
    for (img_path, rgb_img) in rgb_imgs:

        final_boxes, final_cls_inds = pred.inference(rgb_img)
        object_locations = []
        if len(final_boxes):
            for i in range(len(final_boxes)):
                box = final_boxes[i]
                cls = final_cls_inds[i] + 1

                if cls <= 7:
                    x1 = box[0]
                    y1 = box[1]
                    width = box[2] - x1
                    height = box[3] - y1
                    object_locations.append({"type": int(cls), "x": int(x1), "y": int(y1),
                                             "width": int(width), "height": int(height), "segmentation": []})
                if (int(cls) == 8 or int(cls) == 9):
                    x1 = int(box[0])
                    y1 = int(box[1])
                    x2 = int(box[2])
                    y2 = int(box[3])
                    width = x2-x1
                    height = y2-y1
                    # segment approximation simple demo, both left and right leaning, near large far small.
                    # Or you can split dataset to left and right leaning then modify code below
                    object_locations.append({"type": int(cls), "x": -1, "y": -1,
                                             "width": -1, "height": -1,
                                             "segmentation": [
                                                 [x1, y1 + height / 20, x1, y1, x1 + width / 20, y1, x2,
                                                  y2 - height / 20, x2, y2,
                                                  x2 - width / 10, y2],
                                             ]})
                    object_locations.append({"type": int(cls), "x": -1, "y": -1,
                                             "width": -1, "height": -1,
                                             "segmentation": [
                                                 [x1, y1 + height / 15, x1, y1, x1 + width / 15, y1, x2,
                                                  y2 - height / 15, x2, y2,
                                                  x2 - width / 7.5, y2],
                                             ]})
                    object_locations.append({"type": int(cls), "x": -1, "y": -1,
                                             "width": -1, "height": -1,
                                             "segmentation": [
                                                 [x1, y1 + height / 10, x1, y1, x1 + width / 10, y1, x2,
                                                  y2 - height / 10, x2, y2,
                                                  x2 - width / 5, y2],
                                             ]})
                    object_locations.append({"type": int(cls), "x": -1, "y": -1,
                                             "width": -1, "height": -1,
                                             "segmentation": [
                                                 [x1, y1 + height / 7.5, x1, y1, x1 + width / 7.5, y1, x2,
                                                  y2 - height / 7.5, x2, y2,
                                                  x2 - width / 7.5, y2],
                                             ]})

                    object_locations.append({"type": int(cls), "x": -1, "y": -1,
                                             "width": -1, "height": -1,
                                             "segmentation": [
                                                 [x2 - width / 20, y1, x2, y1, x2, y1 + height / 20, x1 + width / 10,
                                                  y2, x1, y2, x1,
                                                  y2 - height / 20],
                                             ]})
                    object_locations.append({"type": int(cls), "x": -1, "y": -1,
                                             "width": -1, "height": -1,
                                             "segmentation": [
                                                 [x2 - width / 15, y1, x2, y1, x2, y1 + height / 15, x1 + width / 7.5,
                                                  y2, x1, y2, x1,
                                                  y2 - height / 15],
                                             ]})
                    object_locations.append({"type": int(cls), "x": -1, "y": -1,
                                             "width": -1, "height": -1,
                                             "segmentation": [
                                                 [x2 - width / 10, y1, x2, y1, x2, y1 + height / 10, x1 + width / 5,
                                                  y2, x1, y2, x1,
                                                  y2 - height / 10],
                                             ]})
                    object_locations.append({"type": int(cls), "x": -1, "y": -1,
                                             "width": -1, "height": -1,
                                             "segmentation": [
                                                 [x2 - width / 7.5, y1, x2, y1, x2, y1 + height / 7.5, x1 + width / 7.5,
                                                  y2, x1, y2, x1,
                                                  y2 - height / 7.5],
                                             ]})

                if (int(cls) == 10):
                    x1 = int(box[0])
                    y1 = int(box[1])
                    x2 = int(box[2])
                    y2 = int(box[3])

                    object_locations.append({"type": int(cls), "x": -1, "y": -1,
                                             "width": -1, "height": -1,
                                             "segmentation": [[x1, y1, x2, y1, x2, y2, x1, y2]]})
        object_locations_by_image[img_path.name] = object_locations

    return object_locations_by_image

## 4. Running Object Detection

Call the following function to run the object detection.  Extra debug output is enabled when `debug` is `True`.

In [4]:
team.run(my_callback, debug=False)

Batch 1 starting. 1 images.
Batch 1 done. Runtime = 0.01830887794494629 seconds.
Batch 2 starting. 1 images.
Batch 2 done. Runtime = 0.017733097076416016 seconds.
Batch 3 starting. 1 images.
Batch 3 done. Runtime = 0.01761031150817871 seconds.
Batch 4 starting. 1 images.
Batch 4 done. Runtime = 0.01799750328063965 seconds.
Batch 5 starting. 1 images.
Batch 5 done. Runtime = 0.017940998077392578 seconds.
Batch 6 starting. 1 images.
Batch 6 done. Runtime = 0.017945051193237305 seconds.
Batch 7 starting. 1 images.
Batch 7 done. Runtime = 0.018232107162475586 seconds.
Batch 8 starting. 1 images.
Batch 8 done. Runtime = 0.018327713012695312 seconds.
Batch 9 starting. 1 images.
Batch 9 done. Runtime = 0.01793074607849121 seconds.
Batch 10 starting. 1 images.
Batch 10 done. Runtime = 0.01875758171081543 seconds.
Batch 11 starting. 1 images.
Batch 11 done. Runtime = 0.018221616744995117 seconds.
Batch 12 starting. 1 images.
Batch 12 done. Runtime = 0.019535064697265625 seconds.
Batch 13 starti

Batch 99 done. Runtime = 0.020115375518798828 seconds.
Batch 100 starting. 1 images.
Batch 100 done. Runtime = 0.017760276794433594 seconds.
Batch 101 starting. 1 images.
Batch 101 done. Runtime = 0.018041610717773438 seconds.
Batch 102 starting. 1 images.
Batch 102 done. Runtime = 0.01864790916442871 seconds.
Batch 103 starting. 1 images.
Batch 103 done. Runtime = 0.01865983009338379 seconds.
Batch 104 starting. 1 images.
Batch 104 done. Runtime = 0.018265724182128906 seconds.
Batch 105 starting. 1 images.
Batch 105 done. Runtime = 0.018280744552612305 seconds.
Batch 106 starting. 1 images.
Batch 106 done. Runtime = 0.017818689346313477 seconds.
Batch 107 starting. 1 images.
Batch 107 done. Runtime = 0.019030094146728516 seconds.
Batch 108 starting. 1 images.
Batch 108 done. Runtime = 0.018055200576782227 seconds.
Batch 109 starting. 1 images.
Batch 109 done. Runtime = 0.018099308013916016 seconds.
Batch 110 starting. 1 images.
Batch 110 done. Runtime = 0.01832294464111328 seconds.
Ba

Batch 197 done. Runtime = 0.017580032348632812 seconds.
Batch 198 starting. 1 images.
Batch 198 done. Runtime = 0.018951892852783203 seconds.
Batch 199 starting. 1 images.
Batch 199 done. Runtime = 0.01780080795288086 seconds.
Batch 200 starting. 1 images.
Batch 200 done. Runtime = 0.01897144317626953 seconds.
Batch 201 starting. 1 images.
Batch 201 done. Runtime = 0.019594192504882812 seconds.
Batch 202 starting. 1 images.
Batch 202 done. Runtime = 0.018743038177490234 seconds.
Batch 203 starting. 1 images.
Batch 203 done. Runtime = 0.01855325698852539 seconds.
Batch 204 starting. 1 images.
Batch 204 done. Runtime = 0.0179903507232666 seconds.
Batch 205 starting. 1 images.
Batch 205 done. Runtime = 0.018929481506347656 seconds.
Batch 206 starting. 1 images.
Batch 206 done. Runtime = 0.018074512481689453 seconds.
Batch 207 starting. 1 images.
Batch 207 done. Runtime = 0.018707990646362305 seconds.
Batch 208 starting. 1 images.
Batch 208 done. Runtime = 0.01835322380065918 seconds.
Batc

Batch 294 done. Runtime = 0.018230915069580078 seconds.
Batch 295 starting. 1 images.
Batch 295 done. Runtime = 0.01755356788635254 seconds.
Batch 296 starting. 1 images.
Batch 296 done. Runtime = 0.018056154251098633 seconds.
Batch 297 starting. 1 images.
Batch 297 done. Runtime = 0.018665790557861328 seconds.
Batch 298 starting. 1 images.
Batch 298 done. Runtime = 0.017918109893798828 seconds.
Batch 299 starting. 1 images.
Batch 299 done. Runtime = 0.017904281616210938 seconds.
Batch 300 starting. 1 images.
Batch 300 done. Runtime = 0.017530202865600586 seconds.
Batch 301 starting. 1 images.
Batch 301 done. Runtime = 0.017995834350585938 seconds.
Batch 302 starting. 1 images.
Batch 302 done. Runtime = 0.022409439086914062 seconds.
Batch 303 starting. 1 images.
Batch 303 done. Runtime = 0.017783403396606445 seconds.
Batch 304 starting. 1 images.
Batch 304 done. Runtime = 0.019028186798095703 seconds.
Batch 305 starting. 1 images.
Batch 305 done. Runtime = 0.01839733123779297 seconds.


Batch 391 starting. 1 images.
Batch 391 done. Runtime = 0.019405364990234375 seconds.
Batch 392 starting. 1 images.
Batch 392 done. Runtime = 0.018236637115478516 seconds.
Batch 393 starting. 1 images.
Batch 393 done. Runtime = 0.017565250396728516 seconds.
Batch 394 starting. 1 images.
Batch 394 done. Runtime = 0.019212722778320312 seconds.
Batch 395 starting. 1 images.
Batch 395 done. Runtime = 0.01764392852783203 seconds.
Batch 396 starting. 1 images.
Batch 396 done. Runtime = 0.01813650131225586 seconds.
Batch 397 starting. 1 images.
Batch 397 done. Runtime = 0.017683029174804688 seconds.
Batch 398 starting. 1 images.
Batch 398 done. Runtime = 0.018397808074951172 seconds.
Batch 399 starting. 1 images.
Batch 399 done. Runtime = 0.021599531173706055 seconds.
Batch 400 starting. 1 images.
Batch 400 done. Runtime = 0.017968177795410156 seconds.
Batch 401 starting. 1 images.
Batch 401 done. Runtime = 0.018238306045532227 seconds.
Batch 402 starting. 1 images.
Batch 402 done. Runtime = 

...

Batch 1555 starting. 1 images.
Batch 1555 done. Runtime = 0.01849508285522461 seconds.
Batch 1556 starting. 1 images.
Batch 1556 done. Runtime = 0.018001794815063477 seconds.
Batch 1557 starting. 1 images.
Batch 1557 done. Runtime = 0.017971038818359375 seconds.
Batch 1558 starting. 1 images.
Batch 1558 done. Runtime = 0.01845526695251465 seconds.
Batch 1559 starting. 1 images.
Batch 1559 done. Runtime = 0.018141984939575195 seconds.
Batch 1560 starting. 1 images.
Batch 1560 done. Runtime = 0.017516374588012695 seconds.
Batch 1561 starting. 1 images.
Batch 1561 done. Runtime = 0.017973661422729492 seconds.
Batch 1562 starting. 1 images.
Batch 1562 done. Runtime = 0.018053293228149414 seconds.
Batch 1563 starting. 1 images.
Batch 1563 done. Runtime = 0.018121719360351562 seconds.
Batch 1564 starting. 1 images.
Batch 1564 done. Runtime = 0.017694950103759766 seconds.
Batch 1565 starting. 1 images.
Batch 1565 done. Runtime = 0.018078327178955078 seconds.
Batch 1566 starting. 1 images.
Bat

Batch 1651 starting. 1 images.
Batch 1651 done. Runtime = 0.017925262451171875 seconds.
Batch 1652 starting. 1 images.
Batch 1652 done. Runtime = 0.017749309539794922 seconds.
Batch 1653 starting. 1 images.
Batch 1653 done. Runtime = 0.01815199851989746 seconds.
Batch 1654 starting. 1 images.
Batch 1654 done. Runtime = 0.018693208694458008 seconds.
Batch 1655 starting. 1 images.
Batch 1655 done. Runtime = 0.018413543701171875 seconds.
Batch 1656 starting. 1 images.
Batch 1656 done. Runtime = 0.017666101455688477 seconds.
Batch 1657 starting. 1 images.
Batch 1657 done. Runtime = 0.017955780029296875 seconds.
Batch 1658 starting. 1 images.
Batch 1658 done. Runtime = 0.018402576446533203 seconds.
Batch 1659 starting. 1 images.
Batch 1659 done. Runtime = 0.019042015075683594 seconds.
Batch 1660 starting. 1 images.
Batch 1660 done. Runtime = 0.019153594970703125 seconds.
Batch 1661 starting. 1 images.
Batch 1661 done. Runtime = 0.017995595932006836 seconds.
Batch 1662 starting. 1 images.
Ba

Batch 1745 done. Runtime = 0.017941713333129883 seconds.
Batch 1746 starting. 1 images.
Batch 1746 done. Runtime = 0.01713275909423828 seconds.
Batch 1747 starting. 1 images.
Batch 1747 done. Runtime = 0.017591238021850586 seconds.
Batch 1748 starting. 1 images.
Batch 1748 done. Runtime = 0.0184175968170166 seconds.
Batch 1749 starting. 1 images.
Batch 1749 done. Runtime = 0.01947808265686035 seconds.
Batch 1750 starting. 1 images.
Batch 1750 done. Runtime = 0.017827749252319336 seconds.
Batch 1751 starting. 1 images.
Batch 1751 done. Runtime = 0.017955780029296875 seconds.
Batch 1752 starting. 1 images.
Batch 1752 done. Runtime = 0.017735719680786133 seconds.
Batch 1753 starting. 1 images.
Batch 1753 done. Runtime = 0.018819093704223633 seconds.
Batch 1754 starting. 1 images.
Batch 1754 done. Runtime = 0.018121957778930664 seconds.
Batch 1755 starting. 1 images.
Batch 1755 done. Runtime = 0.018113374710083008 seconds.
Batch 1756 starting. 1 images.
Batch 1756 done. Runtime = 0.0181126

Batch 1842 starting. 1 images.
Batch 1842 done. Runtime = 0.018452882766723633 seconds.
Batch 1843 starting. 1 images.
Batch 1843 done. Runtime = 0.01802372932434082 seconds.
Batch 1844 starting. 1 images.
Batch 1844 done. Runtime = 0.017838239669799805 seconds.
Batch 1845 starting. 1 images.
Batch 1845 done. Runtime = 0.018423795700073242 seconds.
Batch 1846 starting. 1 images.
Batch 1846 done. Runtime = 0.01847052574157715 seconds.
Batch 1847 starting. 1 images.
Batch 1847 done. Runtime = 0.018990278244018555 seconds.
Batch 1848 starting. 1 images.
Batch 1848 done. Runtime = 0.017634153366088867 seconds.
Batch 1849 starting. 1 images.
Batch 1849 done. Runtime = 0.018697261810302734 seconds.
Batch 1850 starting. 1 images.
Batch 1850 done. Runtime = 0.019120454788208008 seconds.
Batch 1851 starting. 1 images.
Batch 1851 done. Runtime = 0.018001079559326172 seconds.
Batch 1852 starting. 1 images.
Batch 1852 done. Runtime = 0.017811298370361328 seconds.
Batch 1853 starting. 1 images.
Bat

Batch 1938 starting. 1 images.
Batch 1938 done. Runtime = 0.018738985061645508 seconds.
Batch 1939 starting. 1 images.
Batch 1939 done. Runtime = 0.016302824020385742 seconds.
Batch 1940 starting. 1 images.
Batch 1940 done. Runtime = 0.019149303436279297 seconds.
Batch 1941 starting. 1 images.
Batch 1941 done. Runtime = 0.018346071243286133 seconds.
Batch 1942 starting. 1 images.
Batch 1942 done. Runtime = 0.01863718032836914 seconds.
Batch 1943 starting. 1 images.
Batch 1943 done. Runtime = 0.018349170684814453 seconds.
Batch 1944 starting. 1 images.
Batch 1944 done. Runtime = 0.018857717514038086 seconds.
Batch 1945 starting. 1 images.
Batch 1945 done. Runtime = 0.017954587936401367 seconds.
Batch 1946 starting. 1 images.
Batch 1946 done. Runtime = 0.01873612403869629 seconds.
Batch 1947 starting. 1 images.
Batch 1947 done. Runtime = 0.018873929977416992 seconds.
Batch 1948 starting. 1 images.
Batch 1948 done. Runtime = 0.0182650089263916 seconds.
Batch 1949 starting. 1 images.
Batch